In [ ]:
import os
import numpy as np
import librosa
import sounddevice as sd
import tensorflow as tf
import tensorflow_hub as hub
import threading
import cv2
from scipy.io.wavfile import write
from scipy.signal import butter, lfilter
from ultralytics import YOLO
from datetime import datetime
from deepface import DeepFace

# === Model Loading ===
yolo_model_path = 'yolo_v1 (2).pt'  # YOLO model path

# Load models
audio_model = tf.keras.models.load_model("best_model.keras")
yamnet_model = hub.load('https://www.kaggle.com/models/google/yamnet/TensorFlow2/yamnet/1')
yolo_model = YOLO(yolo_model_path)

# Ensure required directories exist
os.makedirs("anomalous_videos", exist_ok=True)
os.makedirs("anomalous_audio", exist_ok=True)
os.makedirs("unknown_faces", exist_ok=True)

# Index-to-label mapping for YAMNet
index_to_label = {
    0: "Emergency_alert_sound",
    1: "Explosions",
    2: "Gunshots",
    3: "Human screams",
    4: "Bottles breaking",
    5: "Dog bark"
    # Add more labels as per your trained model
}

# Noise reduction using a Butterworth filter
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return b, a

def bandpass_filter(data, lowcut=300.0, highcut=3400.0, fs=16000, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

# Preprocess and generate YAMNet embeddings
def preprocess_audio(audio_waveform, target_length=16000 * 5):
    # Apply noise reduction
    audio_waveform = bandpass_filter(audio_waveform)

    # Pad or truncate the waveform to the target length
    if len(audio_waveform) < target_length:
        audio_waveform = np.pad(audio_waveform, (0, target_length - len(audio_waveform)))
    else:
        audio_waveform = audio_waveform[:target_length]

    # Normalize between -1 and 1
    audio_waveform = audio_waveform.astype(np.float32)
    audio_waveform /= np.max(np.abs(audio_waveform))
    return audio_waveform

def predict_audio_from_stream(audio_data, sr, confidence_threshold=0.7):
    processed_audio = preprocess_audio(audio_data)
    _, yamnet_embeddings, _ = yamnet_model(processed_audio)
    avg_embedding = tf.reduce_mean(yamnet_embeddings, axis=0).numpy().reshape(1, -1)

    prediction = audio_model.predict(avg_embedding)
    predicted_class_index = np.argmax(prediction, axis=1)[0]
    confidence = prediction[0][predicted_class_index]

    if confidence >= confidence_threshold:
        predicted_class_name = index_to_label.get(predicted_class_index, "Unknown")
        return predicted_class_name, confidence
    return "Unknown", confidence

# Audio capturing and processing thread
def process_audio_stream(duration=5, sample_rate=16000):
    print("[INFO] Starting audio detection thread...")
    while True:
        print("[INFO] Recording audio...")
        audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='float32')
        sd.wait()  # Wait until the recording is finished
        audio_data = audio_data.flatten()

        predicted_class, confidence = predict_audio_from_stream(audio_data, sample_rate)

        if predicted_class != "Unknown":
            print(f"[{datetime.now()}] Detected audio anomaly: {predicted_class} (Confidence: {confidence:.2f})")
            audio_filename = f"anomalous_audio/{predicted_class}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.wav"
            write(audio_filename, sample_rate, (audio_data * 32767).astype(np.int16))
            print(f"[INFO] Audio saved to: {audio_filename}")

# Face recognition setup
known_faces_dir = "known_faces"
os.makedirs(known_faces_dir, exist_ok=True)
known_encodings = []
known_names = []

# Load known faces
def load_known_faces():
    for name in os.listdir(known_faces_dir):
        person_dir = os.path.join(known_faces_dir, name)
        if not os.path.isdir(person_dir):
            continue

        for file in os.listdir(person_dir):
            file_path = os.path.join(person_dir, file)
            try:
                encoding = DeepFace.represent(img_path=file_path, model_name='Facenet')[0]['embedding']
                known_encodings.append(encoding)
                known_names.append(name)
            except Exception as e:
                print(f"Error loading face {file_path}: {e}")

load_known_faces()

def recognize_faces(frame):
    try:
        detections = DeepFace.find(img_path=frame, db_path=known_faces_dir, model_name='Facenet', enforce_detection=False)
        if detections and not detections[0].empty:  # Check if the list is non-empty and DataFrame is not empty
            return detections[0].iloc[0]['identity']  # Return the first match
        else:
            return "Unknown"
    except Exception as e:
        print(f"Error during face recognition: {e}")
        return "Error"


# Video threat detection
def process_video_stream(video_source=0):
    cap = cv2.VideoCapture(video_source)
    if not cap.isOpened():
        print("Error: Could not open video stream")
        return

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS)) or 30

    out = None
    recording = False

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Warning: Failed to capture frame, skipping...")
            continue

        # Perform face recognition
        face_result = recognize_faces(frame)
        if face_result == "Unknown":
            print(f"[{datetime.now()}] Unknown face detected!")
            unknown_face_path = f"unknown_faces/unknown_{datetime.now().strftime('%Y%m%d_%H%M%S')}.jpg"
            cv2.imwrite(unknown_face_path, frame)
            print(f"[INFO] Unknown face snapshot saved to: {unknown_face_path}")

        # Perform object detection
        results = yolo_model.predict(frame, conf=0.5, verbose=False)
        annotated_frame = results[0].plot()

        anomalies_detected = False
        if results[0].boxes:
            for box in results[0].boxes:
                cls_id = int(box.cls)
                confidence = box.conf.item()
                label = yolo_model.names[cls_id]

                print(f"[{datetime.now()}] Detected: {label} (Confidence: {confidence:.2f})")
                if label in ["violence", "weaponized"]:
                    anomalies_detected = True

                    if not recording:
                        video_filename = f"anomalous_videos/{label}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.mp4"
                        out = cv2.VideoWriter(video_filename, fourcc, fps, (frame_width, frame_height))
                        recording = True
                        print(f"[{datetime.now()}] Recording started: {video_filename}")

                    if out:
                        out.write(frame)
                    break

        if not anomalies_detected and recording:
            print(f"[{datetime.now()}] Anomaly ended. Stopping recording.")
            recording = False
            if out:
                out.release()
                out = None

        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Exit requested. Stopping program.")
            break

    if recording and out:
        out.release()
    cap.release()
    cv2.destroyAllWindows()

# Unified system
def unified_system(video_source=0):
    video_thread = threading.Thread(target=process_video_stream, args=(video_source,))
    # audio_thread = threading.Thread(target=process_audio_stream)

    video_thread.start()
    # audio_thread.start()

    video_thread.join()
    # audio_thread.join()

# Entry point
if __name__ == "__main__":
    unified_system(video_source=0)



[INFO] Starting audio detection thread...
[INFO] Recording audio...
25-01-11 23:18:49 - Searching [[[159 162 155]
  [163 167 159]
  [183 187 178]
  ...
  [234 241 248]
  [234 241 248]
  [234 241 248]]

 [[156 162 157]
  [156 160 153]
  [165 167 158]
  ...
  [233 242 247]
  [234 241 248]
  [234 241 248]]

 [[149 155 150]
  [150 154 148]
  [156 158 151]
  ...
  [232 242 246]
  [232 242 246]
  [232 242 246]]

 ...

 [[198 208 212]
  [198 208 212]
  [198 208 212]
  ...
  [ 26  36  40]
  [ 27  37  41]
  [ 28  38  42]]

 [[198 208 212]
  [198 208 212]
  [198 208 212]
  ...
  [ 26  36  40]
  [ 27  37  41]
  [ 28  38  42]]

 [[198 208 212]
  [198 208 212]
  [198 208 212]
  ...
  [ 26  36  40]
  [ 27  37  41]
  [ 29  40  43]]] in 3 length datastore
25-01-11 23:18:52 - find function duration 2.525392770767212 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
[2025-01-11 23:18:54.829706] Detected audio anomaly: Explosions (Confidence: 0.99)
[INFO] Audio saved to: anomalous_audio/Explosions_20250111